# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json
from citipy import citipy

# Import API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the output_data/cities CSV into a data frame
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Create a list using Lat Lng columns as coordinates
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [4]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'hieght': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(15,25))
# Create the heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=1.5)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
plt.savefig('../images.Humidity_Heatmap.png')
fig


#fig(layout=figurelayout(height='420px')

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow weather criteria to Max Temp < 85 and > 70 degrees
# Humidity > 70% 
# Wind Speed < 10 mph
# Cloudiness of 10 or less
# Drop any cities that don't meet all four criteria

fitting_weather = cities_df.dropna()
fitted_cities = fitting_weather.loc[(fitting_weather['Humidity'] >= 70) & (fitting_weather['Cloudiness'] <= 10)]


fitted_cities


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
40,40,hermanus,9,ZA,1558378761,74,-34.42,19.24,55.00,23.78
55,55,khatanga,10,RU,1558378764,92,71.98,102.47,9.96,10.94
57,57,kolyvan,0,RU,1558378764,87,55.31,82.74,48.20,4.47
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52


In [6]:

last_fitted_cities = fitted_cities.loc[(fitted_cities['Max Temp'] >= 70) & (fitted_cities['Max Temp'] <= 85)]


last_fitted_cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
326,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50
337,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41
391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
402,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = last_fitted_cities

hotel_df
                        

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
326,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50
337,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41
391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
402,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50


In [79]:
# Add Hotel Name as an additional column
hotel_df['Hotel Name'] 
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91,
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,


In [9]:
# Reset Index
indexed_hotel_df = hotel_df.reset_index()
del indexed_hotel_df['index']
indexed_hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
1,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,
2,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91,
3,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
5,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,
6,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,
7,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,
9,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50,


In [52]:
hotel = []

for city in range(len(indexed_hotel_df["City"])):
    

    lat = indexed_hotel_df.loc[city]["Lat"]
    lng = indexed_hotel_df.loc[city]["Lng"]
    

        
    coordinates = f"{lat},{lng}"


    target_coordinates = coordinates
    target_search = "hotel"
    target_radius = 8000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    hotel = response.json()['results'][0]['name']
    indexed_hotel_df.loc[city,'Hotel Name'] = hotel
    

In [50]:
response.json()['results'][0]['name']

'Diu Tent City - Diu, India'

In [53]:
indexed_hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,Central Resort Ltd.
1,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,Hotel National
2,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91,Hotel Xaguate
3,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Canto da Canoa - Apartments in Arraial do Cabo
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,"The Stag , A Unit of Deer Valley Homestay"
5,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,OYO 236 Hotel Edmundo
6,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,Osprey Beach Hotel
7,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,Banyan Tree Cabo Marqués
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,iu Hotel Namibe
9,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50,Carrefour


In [43]:
response = response.json()
print(json.dumps(response, indent=4))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIZvs7zXmQ-5aF14W7OCT-fmce15yULISZ8xOp78oVI5VJ739FWMDAqtOhlXUpjDGCxvHBXiQFBk8cRPW5AkZ0oEmRO7Ub3R6H-f0ZKdsqhHT7KyW-XFJoU842uVss3esJXU0v6Id5MVkQLttR7fWsiqx755rMIFwAfyBeveNbAjkYVCCuSQ7cnyXztNoDO1qGcnhb1bl_nKuS1B3CgTBFeWuKt806VRquEFPwiUEjA14JVXyijqpaJj8CkNN_ebDGZsgiD3ODUJptjwS97wOv6qs-Xtp3O48L8GNDtuahlqXueS0Udh34rFszpRDYa4sAp_-w5szZ_qhhgq-r4-Qp7hl0fWkd2wdX5s6w3gvmLl_MY3UFwBDpDpuW-RUc9hCwMP8l2zCacqYNSCU6KAeslBkyV7T8bwlH8vjA8Ve3oSTpxqFnT3etJt17WmoK1eg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.7021811,
                    "lng": 70.9114907
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.70387207989272,
                        "lng": 70.91277627989273
                    },
                    "southwest": {
                        "lat": 20.7011724

In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# Add marker layer ontop of heat map
coordinates = (locations)

# Display figure
print (coordinates)

       Lat     Lng
20   20.86   92.31
87  -13.19   49.05
118  14.90  -24.50
125 -22.97  -42.02
139  15.34   74.49
268  11.00  122.67
326  21.46  -71.14
337  16.86  -99.88
391 -15.19   12.15
402 -14.27   50.17
412  20.71   70.98


In [80]:
figure_layout = {
    'width': '900px',
    'hieght': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(15,25))

In [81]:
# Assign markersalyer to variable
markers = gmaps.marker_layer(coordinates)
    # Add the layer to the map


fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'hieght': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(15,25))
# Create the heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=1.5)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
plt.savefig('../images.hotel_marker.png')
fig